In [ ]:
!pip install openai
!pip install cohere
!pip install tiktoken
!pip install chromadb
!pip install --upgrade langchain
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import openai
import os
import sys
sys.path.append('../..')
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.document_loaders import BSHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate


%env OPENAI_API_KEY="Your-OWN-KEY"
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
def loader(file):
  # load documents
    loader = BSHTMLLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(separators=['\n\n','\n'], chunk_size=4000, chunk_overlap=200, length_function=len)
    docs = text_splitter.split_documents(documents)
    return docs


In [ ]:
docs = loader('testfile1.html')

In [ ]:
docs

[Document(page_content='One plus One is Not Innovation\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSee our response to COVID-19\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBlog\n\n\n\n\n\n\n\n\n\n\n\n\nMenu\n\n\n\n\nPalo Alto Networks\n\nNetwork Security\n\n\nSASE\n\n\nCloud Native Security\n\n\nSecurity Operations\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOne plus One is Not Innovation\n\n\nFirewall\nFirewall \n\n\n\n\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t2,730\n\t\t\t people reacted\n\n\n\n\n\n\n0\n\n\n\n\n\n\n\n\n\n\n\nBy Nir Zuk\nFebruary 3, 2009 at 8:08 PM\n                      1 min. read\n                  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSHARE\n\n\n\n\n\n\n\n\n\n\n\nCheck out Richard Stiennon’s blog entry on his Threat Chaos blog.  I think very highly of Richard – he has never been afraid, even as a Gartner analyst, to say what he thinks even if some of G

In [ ]:
prompt_template_string = """
Your role as a security researcher involves extracting Indicators of Compromise (IoCs) from parsed HTML files.
You focus on specific IoC categories, label the IoC itself with the specific tag given as IoC type : tag as followed
Malicious IP Addresses: <ip>,
Malicious Domain Names: <domain_name>,
Malicious URLs: <url>,
Hash Values: <hash_val>,
Email Addresses: <email>,
File Paths: <file_path>,
Registry Keys: <reg_key>,
Malware Names: <malware>,
Actor/Campaign Names that lauches the attacks: <actor>,
and Geo-locations that launch the attack or being the target of the attack: <geo_loc>.
When an IoC is identified, it's tagged with its actual type, like <ip> IP IoC</ip> or
<geo_location> geo_location IoC</geo_location>, including 50 tokens before and after it from the original article.
This mapping provides context for each IoC, with a total of 100 words surrounding it.
Don't just give examples, you need to list all IoCs from the article.
Your output will follow this markdown format :
### IoC type - IoC itself
#### Context
... tokens before IoC in the origin article <actual IoC type> IoC </actual IoC type> words after IoC in the origin article ...
This approach ensures a comprehensive understanding of the security threat in its original context.

An example is:
give an article " In order to earn money, the Adam group try to use ransomeware to attack user with windows system.
The required credentials depends on the type of data, platform, and access methodology of your app.
There are three types of credential types available"
The IoC we are aiming is the actor/campaign Adam group.
Output markdown format should be

### Actor/Campaign - Adam group
#### Context
In order to earn money, the <actor> Adam group <actor> try to use ransomeware to attack user with windows system.

The input article is:
{input}
"""
# chatgpt = ChatOpenAI(api_key="Your-own-key", model="gpt-4")
# prompt = PromptTemplate.from_template(prompt_template_string)
# output_resp = chatgpt.invoke(prompt.format(input=[element.page_content for element in data])).content

In [ ]:
import os

directory = '/content'
processing_list = []
# Iterating over the files
for filename in os.listdir(directory):
    if filename.endswith('.html'):
        filepath = os.path.join(directory, filename)
        print("Processing file:", filepath)
        processing_list.append(filepath)


Processing file: /content/output_727.html
Processing file: /content/output_784.html
Processing file: /content/output_722.html
Processing file: /content/output_759.html
Processing file: /content/output_853.html
Processing file: /content/output_206.html
Processing file: /content/output_350.html
Processing file: /content/output_689.html
Processing file: /content/output_295.html
Processing file: /content/output_292.html
Processing file: /content/output_706.html
Processing file: /content/output_259.html
Processing file: /content/output_372.html
Processing file: /content/output_497.html
Processing file: /content/output_800.html
Processing file: /content/output_874.html
Processing file: /content/output_179.html
Processing file: /content/output_954.html
Processing file: /content/output_404.html
Processing file: /content/output_855.html
Processing file: /content/output_958.html
Processing file: /content/output_380.html
Processing file: /content/output_905.html
Processing file: /content/output_7

In [ ]:
len(processing_list)

127

In [ ]:
from google.colab import files
import time

processing_list = ['testfile1.html']

for file in processing_list:
  filepath = file
  filename = filepath.split("/")[-1].replace(".html", "")
  print("currently processing {}".format(filename))
  if filename != "output_727":
    data = loader(filepath)
    chatgpt = ChatOpenAI(api_key="YOU-OWN-KEY", model="gpt-4")
    prompt = PromptTemplate.from_template(prompt_template_string)
    output_resp = chatgpt.invoke(prompt.format(input=[element.page_content for element in data])).content
    # Step 1: Redirect print output to a file
    name = "labelled_{}.md".format(filename)
    with open(name, 'w') as f:
        print(output_resp, file=f)
        # Add any other print statements or output here

    # Step 2: Download the file
    files.download(name)
    #allow some time for downloading
    print("Sleep for allowing downloading time ... ")
    time.sleep(45)

currently processing output_725


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Sleep for allowing downloading time ... 


In [ ]:
from google.colab import files
import time
import os
directory = '/content'
processing_list = []
# Iterating over the files
for filename in os.listdir(directory):
  if filename.endswith('.html'):
    filepath = os.path.join(directory, filename)
    # print("Processing file:", filepath)
    processing_list.append(filepath)
for i in range(len(processing_list)):
  element = processing_list[i]
  print(element)
  docs = loader(element)
  input=[element.page_content for element in docs]
  name = "parsed_{}.txt".format(i)
  with open(name, 'w') as f:
      print(input, file=f)
  files.download(name)
  time.sleep(45)

/content/2012:12:27:win32gapz-new-bootkit-technique.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/content/2012:10:03:warning-your-pop-up-ads-may-be-fraudulent-ftc-wins-163m-settlement-against-scareware-firm.html


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/content/2012:12:07:autorun-worm-continues-to-turn.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>